<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Monthly-crime-rates" data-toc-modified-id="Monthly-crime-rates-0.0.1">Monthly crime rates</a></span><ul class="toc-item"><li><span><a href="#Only-blocks-which-are-in-at-least-one-year-treated-or-one,-two,-or-three-cells-over" data-toc-modified-id="Only-blocks-which-are-in-at-least-one-year-treated-or-one,-two,-or-three-cells-over-0.0.1.1">Only blocks which are in at least one year treated or one, two, or three cells over</a></span></li></ul></li><li><span><a href="#Yearly-crime-counts" data-toc-modified-id="Yearly-crime-counts-0.0.2">Yearly crime counts</a></span></li></ul></li><li><span><a href="#Replication-of-descriptives" data-toc-modified-id="Replication-of-descriptives-0.1">Replication of descriptives</a></span><ul class="toc-item"><li><span><a href="#Figure-A.2" data-toc-modified-id="Figure-A.2-0.1.1">Figure A.2</a></span></li><li><span><a href="#Table-1" data-toc-modified-id="Table-1-0.1.2">Table 1</a></span></li><li><span><a href="#Table-2" data-toc-modified-id="Table-2-0.1.3">Table 2</a></span></li><li><span><a href="#Figure-3" data-toc-modified-id="Figure-3-0.1.4">Figure 3</a></span></li></ul></li><li><span><a href="#Estimation" data-toc-modified-id="Estimation-0.2">Estimation</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Empirical-Strategy" data-toc-modified-id="Empirical-Strategy-0.2.0.1">Empirical Strategy</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-0.3">Results</a></span></li></ul></li><li><span><a href="#Additional-figures-for-website" data-toc-modified-id="Additional-figures-for-website-1">Additional figures for website</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-1.1">Load data</a></span><ul class="toc-item"><li><span><a href="#Blocks-with-crimes" data-toc-modified-id="Blocks-with-crimes-1.1.1">Blocks with crimes</a></span></li><li><span><a href="#Routes" data-toc-modified-id="Routes-1.1.2">Routes</a></span></li><li><span><a href="#Hourly-crime-counts" data-toc-modified-id="Hourly-crime-counts-1.1.3">Hourly crime counts</a></span></li></ul></li><li><span><a href="#Plot-blocks-with-dummies" data-toc-modified-id="Plot-blocks-with-dummies-1.2">Plot blocks with dummies</a></span></li><li><span><a href="#Plot-violent-crime-trends" data-toc-modified-id="Plot-violent-crime-trends-1.3">Plot violent crime trends</a></span><ul class="toc-item"><li><span><a href="#Hourly-counts" data-toc-modified-id="Hourly-counts-1.3.1">Hourly counts</a></span></li></ul></li></ul></li></ul></div>

**Description**: This notebook creates all figures and results used on the website [Chicago's Safe Passage Program to Prevent Crime: Is It Worth the Dime?](https://binste.github.io/chicago_safepassage_evaluation/). Some of the figures, descriptive statistics and the main results are a replication of the census block results from [McMillen et al. (2017)](https://ignaciomsarmiento.github.io/assets/Safe_Passage_WP.pdf). Therefore, the first part of the notebook will make direct comparisons to the relevant parts of the beforementioned paper. The second part produces additional figures for the website which are unrelated to McMillen et al. (2017).

In [1]:
import pickle
import sys
from pathlib import Path

import geopandas as gpd
import pandas as pd
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

Set path to data folder

In [3]:
project_folder = Path('../..')
data_path = project_folder / 'data'

### Monthly crime rates

#### Only blocks which are in at least one year treated or one, two, or three cells over

In [9]:
with (data_path / 'processed/est_df.pkl').open('rb') as f:
    est_df = pickle.load(f)
est_df.head()

,tract_bloc,school_year,Date,violent_count,property_count,route_number,school_name,treated,one_over,two_over,three_over,info,time_fe
0,208011000.0,SY0506,2006-01-31,0.0,0.0,NaN,NaN,0.0,0,0,0,-,200601
1,208011000.0,SY0506,2006-02-28,0.0,0.0,NaN,NaN,0.0,0,0,0,-,200602
2,208011000.0,SY0506,2006-03-31,0.0,0.0,NaN,NaN,0.0,0,0,0,-,200603
3,208011000.0,SY0506,2006-04-30,0.0,0.0,NaN,NaN,0.0,0,0,0,-,200604
4,208011000.0,SY0506,2006-05-31,0.0,1.0,NaN,NaN,0.0,0,0,0,-,200605


According to Table 10 in McMillen et al. (2017), they had a sample size of 783,340 for the main specification. It is unclear, why I got more than double the observations.

In [10]:
est_df['school_year'].unique()

array(['SY0506', 'SY0607', 'SY0708', 'SY0809', 'SY0910', 'SY1011',
       'SY1112', 'SY1213', 'SY1314', 'SY1415', 'SY1516'], dtype=object)

## Estimation

#### Empirical Strategy
$$
crimes_{it} = \beta \textit{ treated_block}_{it} + \gamma_1 \textit{ one_block_over}_{it} + \gamma_2 \textit{ two_block_over}_{it} + \delta_i + \lambda_t + e_{it}
$$

* $crimes_{it}$: either the is the monthly violent or the property crime count at school times
* $treated\_block_{it}$ is an indicator variable taking one for blocks in the months that are guarded by Safe Passage personnel
* $one\_block\_over_{it}$, $two\_block\_over_{it}$: indicators for the months after the Safe Passage was enacted if the blocks are one or two blocks over.

Fixed effects:

* $\delta_i$: cell fixed effects
* $\lambda_t$: time fixed effects

Model:

* Poisson regression
    * "The interpretation of a difference-in-difference coefficient from a Poisson regression is $\exp(9) − 1$. However, note that for small enough $\beta$, the approximation $\exp(\beta) − 1 \approx \beta$ is valid."
    
Period of analysis:

* January 2006 - August 2016

Standard errors clustered by:

* Blocks (McMillen et al. cluster at level of Safe Passage routes)

The estimation is done using the R package [glmmML](https://cran.r-project.org/web/packages/glmmML/glmmML.pdf). As it takes quite long to fit the models, they are already supplied in the `../../models/` folder. One with violent crime count as dependent variable, and the other one with property crime counts. If you want to rerun the estimation (might not work if you run this notebook on mybinder.org), set the following parameter to `True`.

In [24]:
rerun_estimations = False

In [25]:
if rerun_estimations:
    # Load packages
    tidyverse = importr('tidyverse')
    glmmML = importr('glmmML')

    # Define functions
    robjects.r("""summary_df <- function(model){
      vars <- names(glm_model_violent$coefficients)[1:3]
      coef <- as.vector(model$coefficients[1:3])
      se <- as.vector(model$sd[1:3])
      z <- coef/se
      p <- 1 - pchisq((z)^2, 1)
      n <- glm_model_violent$n
      return(tibble(var = vars, coef = coef, se = se, z = z, p = p, n = n))
    }

    fit_poisson <- function(df, count_var){
      df$count <- df[[count_var]]
      # First drop all blocks with constant crime count
      # as level would be taken out by tract bloc fixed effects anyway
      non_constant <- df %>%
        group_by(tract_bloc) %>%
        summarize(sd_count = sd(count)) %>%
        filter(sd_count > 0)

      df_nc <- df %>%
        filter(tract_bloc %in% non_constant$tract_bloc) %>%
        select(count, treated, one_over, two_over, tract_bloc, time_fe)
      df_nc$tract_bloc <- as.factor(df_nc$tract_bloc)

      # Fit model
      print(paste0("Fit model with ", count_var))
      poisson_model <- glmmboot(count ~ treated + one_over + two_over + factor(time_fe),
                                    family = poisson, data = df_nc, cluster = tract_bloc)
      return(poisson_model)
    }
    """)

    # Load data
    print('Convert Python data to R')
    pandas2ri.activate()
    robjects.globalenv['est_df'] = pandas2ri.py2ri(est_df[[
        'tract_bloc', 'violent_count', 'property_count', 'treated', 'one_over',
        'two_over', 'three_over', 'time_fe'
    ]])
    # Fit poisson model for violent count
    robjects.r('glm_model_violent <- fit_poisson(est_df, "violent_count")')
    robjects.r('saveRDS(glm_model_violent, "../../models/poisson_violent.rds")')
    # Save summary
    robjects.r('summary_violent <- summary_df(glm_model_violent)')
    robjects.r('write_csv(summary_violent, "../../models/summary_poisson_violent.csv")')
    # Fit poisson model for property count
    robjects.r('glm_model_property <- fit_poisson(est_df, "property_count")')
    robjects.r('saveRDS(glm_model_property, "../../models/poisson_property.rds")')
    # Save summary
    robjects.r('summary_property <- summary_df(glm_model_property)')
    robjects.r('write_csv(summary_property, "../../models/summary_poisson_property.csv")')